In [1]:
import pandas as pd
import numpy as np

def BinomialTree (type,S0, K, r, sigma, T, N=2000,american="false"):

    #calculate delta T    
    deltaT = float(T) / N

    # up and down factor will be constant for the tree so we calculate outside the loop
    u = np.exp(sigma * np.sqrt(deltaT))
    d = 1.0 / u

    #initializing array
    fs =  np.asarray([0.0 for i in range(N + 1)])

    #stock values values at expiration 
    fs2 = np.asarray([(S0 * u**j * d**(N - j)) for j in range(N + 1)])

    #we vectorize the strikes as well so the expiration check will be faster
    fs3 =np.asarray( [float(K) for i in range(N + 1)])

    #uptick and downtick probabilties 
    a = np.exp(r * deltaT)
    p = (a - d)/ (u - d)
    oneMinusP = 1.0 - p


    # Compute the leaves, f_{N, j}
    if type =="C":
        fs[:] = np.maximum(fs2-fs3, 0.0)
    else:
        fs[:] = np.maximum(-fs2+fs3, 0.0)


    #calculate backward the option prices
    for i in range(N-1, -1, -1):
       fs[:-1]=np.exp(-r * deltaT) * (p * fs[1:] + oneMinusP * fs[:-1])
       fs2[:]=fs2[:]*u

       if american=='true':
           #Simply check if the option is worth more alive or dead
           if type =="C":
                fs[:]=np.maximum(fs[:],fs2[:]-fs3[:])
           else:
                fs[:]=np.maximum(fs[:],-fs2[:]+fs3[:])

    # print fs
    return fs[0]

In [2]:
print (BinomialTree("C",55,50,0.08,0.1,T=1,american="true"))

8.92498687278143
